In [1]:
import pandas as pd

In [2]:
df_metadata = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')

In [3]:
df_metadata = df_metadata[['category', 'attribute_field', 'category_attributevalue', 'description', 'max_multi_select', 'usage', 'entry mode']]

In [4]:
metadata_category = set(df_metadata.category)

In [5]:
category2config = {}
for i in metadata_category:
    df_tmp = df_metadata[df_metadata.category == i]
    category2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])

In [9]:
df_samples = pd.read_csv('allstratified_sample_4806_query.csv').rename(columns={'top_query_classification_taxonomy': 'category_path'})

In [13]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: any([i.startswith(x) for i in metadata_category]))]
df_samples = df_samples.sample(10, random_state=42)

In [16]:
df_samples

,label_ordering,query,sample_method,category_path
2231,44317,memory card ps2,head,Consumer Electronics > Video Games > Memory Cards
3246,66149,cat tower for indoor cats,head,Home & Garden > Pet Products > Cat Supplies > ...
2947,60076,aprons with pockets,uniform,Home & Garden > Household Cleaning > Household...
1721,33327,abafador eletronico para tiro,head,Apparel Accessories > Girls' Accessories > Gir...
793,15276,winter hats for men,head,Apparel Accessories > Men's Hats > Men's Bombe...
3092,62891,camping toilets portable,head,Home Improvement > Bathroom Fixtures > Toilets...
4415,91164,foto box studio,uniform,Consumer Electronics > Camera & Photo > Photo ...
3272,66607,shoe rack organizer,head,Home & Garden > Home Storage & Organization > ...
826,15766,costume bagno uomo,head,Sports > Swimming > Men's Swimwear > Men's Bod...
1617,31260,machine a popcorn,uniform,Home Improvement > Home Appliances > Kitchen A...


In [18]:
from functools import reduce

In [20]:
reduce(lambda x,y: x + y, *[[], [2]])

[2]

In [23]:
from copy import deepcopy

In [24]:
res = []
for i in df_samples[['query', 'sample_method', 'category_path']].to_dict('records'):
    i['text'] = f"Query: {i['query']}"
    attr_config = []
    for j in metadata_category:
        if j.startswith(i['category_path']):
            attr_config += deepcopy(category2config[j])
    i['attribute_config'] = attr_config
    res.append(i)

In [27]:
pd.DataFrame(res).to_json('example_offshore_input_query_attr_extract.json', lines=True, orient='records')